In [1]:
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, AutoencoderKL
from diffusers import DDPMScheduler
from diffusers import StableDiffusionPipeline
from transformers import CLIPTextModel  # <-- from transformers, not diffusers
from transformers import CLIPTextModelWithProjection, CLIPTokenizer
import torch

c:\Users\Aishu\anaconda3\envs\eoi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from typing import List
import random, math, os, sys
import numpy as np
from tqdm import tqdm
from timm.utils import ModelEmaV3
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim

In [3]:
root_directory = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(root_directory, 'src'))

In [4]:
from text_encoders import clip_model, bert_model
from dataloaders_text import caption_dataset

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0").to(device)


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is 

In [6]:
from accelerate import Accelerator

accelerator = Accelerator()
pipe.enable_attention_slicing()

In [7]:
training_args = {
    "learning_rate": 1e-5,  # A smaller learning rate works best for fine-tuning
    "batch_size": 8,        # Adjust based on your GPU's VRAM
    "gradient_accumulation_steps": 4,  # Helps manage larger batch sizes with limited VRAM
    "num_train_epochs": 5,  # Enough to converge without overfitting
}

In [8]:
dataloader = caption_dataset()
train_dataloader,_ = dataloader.get_dataloader(partition="train", batch_size=32)

Error loading Flickr CSV: [Errno 2] No such file or directory: 'c:\\Users\\Aishu\\Downloads\\AML-2\\Echoes-of-Imagination\\data\\datas\\extracted_files\\flickr30k_images/results.csv'. Using a small dummy set for Flickr.


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Aishu\\Downloads\\AML-2\\Echoes-of-Imagination\\data\\datas\\extracted_files\\annotations_trainval2014/annotations/captions_train2014.json'